In [1]:
import pandas as pd
import numpy as np
import tabula
import glob

In [27]:
def sber_pdf_to_csv(credit=False):
    
    """
    Функция предназначена для обработки отчетов по картам из личного кабинета в сбербанке.
    Отчеты по картам из сбербанка находятся в формате pdf.
    Считывает файлы pdf из корневой папки.
    P.S. Отчеты по кредитным картам нужно обрабатывать отдельно
    
    В функции используется модуль tabula для парсинга pdf файлов.
    
    Так как отчеты по кредитной карте идут немного в другом формате. Был добавлен параметр 'credit'
    Параметр "credit" (по умолчанию False)
    Для того что бы обрабатывать отчеты по кредитным картам, параметр credit нужно указать True 
    
    """
    pdf = glob.glob('*.pdf')
    df_sample = pd.DataFrame()
    counter_card = 0
    
    for name in pdf:
        counter_card += 1
        df_sample.append(df_sample)
        
        
        pdf_table = tabula.read_pdf(name,
                               pages='all',
                               multiple_tables=True)

        for table in pdf_table:

            if credit is False:
                df = table[2:].copy()
                df.columns = ['date', 'category', 'amount', 'remains']
            else:
                table = table.reset_index().T.reset_index().T.reset_index(drop=True).drop(columns=0)
                table['date'] = table[1].str.cat(table[2], sep =" ")
                df = table[['date',3,4]].copy()
                df.columns = ['date', 'category', 'amount']

            df['amount'] = df['amount'].str.replace(',','.')
            df['amount'] = df['amount'].str.replace(' ','')
            df['amount'] = df['amount'].astype(float)

            if credit is False:
                df['remains'] = df['remains'].str.replace(',','.')
                df['remains'] = df['remains'].str.replace(' ','')
                df['remains'] = df['remains'].astype(float)


            l = 0 - len(df)
            desc = []
            a = np.array(df.amount)
            b = np.array(df.category)

            while l < 1: 
                if np.isnan(a[l]) == True and np.isnan(a[l+1]) == True:
                    desc.append(b[l] + " " + b[l+1])
                    l+=1
                elif np.isnan(a[l+1]) == False and np.isnan(a[l]) == True and np.isnan(a[l-1]) == False:
                    desc.append(b[l])
                    l+=1
                else:
                    l+=1

            df = df.dropna()

            if len(df) == len(desc):
                df['description'] = desc    
            else:
                desc.append('unknown')
                df['description'] = desc
                
            df['card'] = counter_card
            df_sample = pd.concat([df_sample, df])
            

    df_sample['date'] = pd.to_datetime(df_sample['date'])
    df_sample.sort_values('date', ascending=True)
    df_sample.reset_index(drop=True, inplace=True)

    return df_sample

In [28]:
df = sber_pdf_to_csv()

C:\Users\smars\AppData\Local\Temp\ipykernel_26320\3243635097.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sample.append(df_sample)
C:\Users\smars\AppData\Local\Temp\ipykernel_26320\3243635097.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sample.append(df_sample)


In [31]:
df.head()

,date,category,amount,remains,description,card
0,2022-05-21 19:40:00,Транспорт,28.00,13453.46,SITICARD. TRANS,1
1,2022-05-21 18:31:00,Отдых и развлечения,350.00,13481.46,PARK 2,1
2,2022-05-21 12:32:00,Рестораны и кафе,200.00,13831.46,ARZA OOO GORIZONTY VKUSA,1
3,2022-05-21 11:18:00,Супермаркеты,44.99,14031.46,PYATEROCHKA 9488,1
4,2022-05-21 06:56:00,Прочие операции,6.00,14076.45,SBERBANK ONL@IN KARTA-VKLAD,1
